In this example, we will use tensorflow.keras package to create a keras image classification application using model MobileNetV2, and transfer the application to Cluster Serving step by step.

In [2]:
import tensorflow as tf
import os
import PIL

In [3]:
tf.__version__

'2.2.0'

In [4]:
# Obtain data from url:"https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip",
                                   fname="cats_and_dogs_filtered.zip", extract=True)

# Find the directory of validation set
base_dir, _ = os.path.splitext(zip_file)
test_dir = os.path.join(base_dir, 'validation')
# Set images size to 160x160x3
image_size = 160

# Rescale all images by 1./255 and apply image augmentation
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow images using generator to the test_generator
test_generator = test_datagen.flow_from_directory(
                test_dir,
                target_size=(image_size, image_size),
                batch_size=1,
                class_mode='binary')

Found 1000 images belonging to 2 classes.


In [5]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE=(160,160,3)
model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In keras, input could be ndarray, or generator. Here, we input the generator to model.

In [7]:
prediction=model.predict(test_generator)
print(prediction)

[[[[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.26466656 ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         1.2028642
    0.25536

Great! Now the Keras application is completed. We are now transferring it to Cluster Serving. The first step is to save the model to SavedModel format.

In [9]:
# Save trained model to ./transfer_learning_mobilenetv2
model.save('transfer_learning_mobilenetv2')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: transfer_learning_mobilenetv2/assets


Then, we config the model path in `config.yaml` (if you do not know what is `config.yaml`, check [Cluster Serving Configuration](https://github.com/intel-analytics/analytics-zoo/blob/master/docs/docs/ClusterServingGuide/ProgrammingGuide.md#2-configuration))

After configuration, start Cluster Serving by `cluster-serving-start` as mentioned in [Cluster Serving Programming Guide](https://github.com/intel-analytics/analytics-zoo/blob/master/docs/docs/ClusterServingGuide/ProgrammingGuide.md#3-launching-service)

Next we start Cluster Serving code

In [ ]:
from zoo.serving.client import InputQueue, OutputQueue
input_queue = InputQueue()

In Cluster Serving, only ndarray is supported as input. Thus, we first transform the generator to ndarray

In [11]:
arr = test_generator.next()[0]
arr

array([[[[0.04705883, 0.        , 0.07450981],
         [0.06666667, 0.05882353, 0.21568629],
         [0.57254905, 0.68235296, 0.8705883 ],
         ...,
         [0.5176471 , 0.38823533, 0.21960786],
         [0.4784314 , 0.3529412 , 0.19215688],
         [0.48627454, 0.37254903, 0.20784315]],

        [[0.12941177, 0.09803922, 0.25882354],
         [0.38431376, 0.37647063, 0.52156866],
         [0.8862746 , 0.9215687 , 0.9803922 ],
         ...,
         [0.4666667 , 0.36862746, 0.21176472],
         [0.45882356, 0.3647059 , 0.21568629],
         [0.45098042, 0.3647059 , 0.21960786]],

        [[0.4901961 , 0.52156866, 0.5294118 ],
         [0.54901963, 0.5529412 , 0.52156866],
         [0.5294118 , 0.49803925, 0.35686275],
         ...,
         [0.41960788, 0.34509805, 0.21960786],
         [0.43529415, 0.37254903, 0.24313727],
         [0.4156863 , 0.35686275, 0.23529413]],

        ...,

        [[0.34901962, 0.47450984, 0.57254905],
         [0.37647063, 0.5019608 , 0.6       ]

In [ ]:
# Use async api to put and get, you have pass a name arg and use the name to get
input_queue.enqueue('my-input', arr)
output_queue = OutputQueue()
prediction = output_queue.query('my-input')
# Use sync api to predict, this will block until the result is get or timeout
prediction = input_queue.predict(arr)

This is the end of this tutorial. If you have any question, you could raise an issue at [Analytics Zoo Github](https://github.com/intel-analytics/analytics-zoo/issues).